In [1]:
import sys

sys.path.append('..')

from action_language_interpreter.action_language import *

In [2]:
events: Set[Event] = {"wait", "move"}

In [3]:
p0 = FluentLiteral("position(0)")
p1 = FluentLiteral("position(1)")
p2 = FluentLiteral("position(2)")
b0 = FluentLiteral("battery(0)")
b1 = FluentLiteral("battery(1)")
b2 = FluentLiteral("battery(2)")
o = FluentLiteral("objective_reached")

## Manual Grounding

In [4]:
AD = ActionDescription()
AD.add_stmt_impossible_if("move", b0) # Cannot move if battery is empty
AD.add_stmt_impossible_if("move", p2) # Cannot move once in the goal
# Moving implies that the robot is no longer in position
AD.add_stmt_causes_if("move", ~p0, p0)
AD.add_stmt_causes_if("move", ~p1, p1)
AD.add_stmt_causes_if("move", ~p2, p2)
# Moving or waiting implies that the battery level has changed
AD.add_stmt_causes_if("move", ~b1, b1)
AD.add_stmt_causes_if("move", ~b2, b2)
AD.add_stmt_causes_if("wait", ~b1, b1)
AD.add_stmt_causes_if("wait", ~b2, b2)
# The battery reduces by for either moving or waiting
AD.add_stmt_causes_if("wait", b0, b0)
AD.add_stmt_causes_if("move", b0, b1)
AD.add_stmt_causes_if("wait", b0, b1)
AD.add_stmt_causes_if("move", b1, b2)
AD.add_stmt_causes_if("wait", b1, b2)
# Moving changes the position by +1
AD.add_stmt_causes_if("move", p1, p0)
AD.add_stmt_causes_if("move", p2, p1)
# Waiting does not change the position
AD.add_stmt_causes_if("wait", p0, p0)
AD.add_stmt_causes_if("wait", p1, p1)
AD.add_stmt_causes_if("wait", p2, p2)
# The positions are mutually exclusive
AD.add_stmt_if(~p0, p1)
AD.add_stmt_if(~p0, p2)
AD.add_stmt_if(~p1, p0)
AD.add_stmt_if(~p1, p2)
AD.add_stmt_if(~p2, p0)
AD.add_stmt_if(~p2, p1)
# The battery states are mutually exclusive
AD.add_stmt_if(~b0, b1)
AD.add_stmt_if(~b0, b2)
AD.add_stmt_if(~b1, b0)
AD.add_stmt_if(~b1, b2)
AD.add_stmt_if(~b2, b0)
AD.add_stmt_if(~b2, b1)
# The objective is not reached if not on position 2
AD.add_stmt_if(~o, ~p2)
# The objective is reached if on position 2
AD.add_stmt_if(o, p2, ~o)
AD

({move causes ¬battery(2) if battery(2), wait causes ¬battery(2) if battery(2), move causes ¬battery(1) if battery(1), move causes position(2) if position(1), wait causes ¬battery(1) if battery(1), wait causes battery(0) if battery(1), move causes ¬position(2) if position(2), wait causes position(2) if position(2), move causes ¬position(1) if position(1), wait causes position(1) if position(1), move causes ¬position(0) if position(0), wait causes battery(0) if battery(0), move causes position(1) if position(0), move causes battery(1) if battery(2), wait causes battery(1) if battery(2), wait causes position(0) if position(0), move causes battery(0) if battery(1)}, {¬position(1) if position(2), ¬battery(1) if battery(0), ¬position(2) if position(1), ¬battery(2) if battery(1), ¬position(0) if position(1), ¬position(0) if position(2), ¬position(1) if position(0), ¬battery(0) if battery(2), ¬position(2) if position(0), ¬battery(1) if battery(2), ¬objective_reached if ¬position(2), ¬battery(

# Execution

In [5]:
initial_state = {p0, b2}
initial_state

{battery(2), position(0)}

In [6]:
AD.comply(initial_state)

({¬battery(0),
  ¬battery(1),
  battery(2),
  ¬objective_reached,
  position(0),
  ¬position(1),
  ¬position(2)},
 [¬position(1) if position(0),
  ¬battery(0) if battery(2),
  ¬position(2) if position(0),
  ¬battery(1) if battery(2),
  ¬objective_reached if ¬position(2)])

In [7]:
AD.executable_events(initial_state, events)

({'move', 'wait'}, ())

## Successfull Route

In [8]:
state1,relevant_stmt1 = AD.execute(initial_state, "move")

In [9]:
state1

{¬battery(0),
 battery(1),
 ¬battery(2),
 ¬objective_reached,
 ¬position(0),
 position(1),
 ¬position(2)}

In [10]:
relevant_stmt1

[move causes ¬battery(2) if battery(2),
 move causes ¬position(0) if position(0),
 move causes position(1) if position(0),
 move causes battery(1) if battery(2),
 ¬position(2) if position(1),
 ¬battery(2) if battery(1),
 ¬position(0) if position(1),
 ¬objective_reached if ¬position(2),
 ¬battery(0) if battery(1)]

In [11]:
AD.executable_events(state1, events)

({'move', 'wait'}, ())

In [12]:
state2, relevant_stmt2 = AD.execute(state1, "move")

In [13]:
state2

{battery(0),
 ¬battery(1),
 ¬battery(2),
 objective_reached,
 ¬position(0),
 ¬position(1),
 position(2)}

In [14]:
relevant_stmt2

[move causes ¬battery(1) if battery(1),
 move causes position(2) if position(1),
 move causes ¬position(1) if position(1),
 move causes battery(0) if battery(1),
 ¬position(1) if position(2),
 ¬battery(1) if battery(0),
 ¬position(0) if position(2),
 ¬battery(2) if battery(0),
 objective_reached if position(2),¬objective_reached]

In [15]:
AD.executable_events(state2, events)

({'wait'}, (move impossible_if position(2), move impossible_if battery(0)))

## Explanation

In [16]:
scenario_path = (initial_state,[("move", state1),("move", state2)])

## First

In [17]:
direct_effects, indirect_effects = get_first_causal_explanation(action_description=AD, scenario_path=scenario_path, outcome={o}, transition_event="move")

In [18]:
direct_effects

{battery(0), ¬battery(1), ¬position(1), position(2)}

In [19]:
indirect_effects

{objective_reached}

## Second

In [20]:
supporting_literals, initial_literals = get_second_causal_explanation(scenario_path=scenario_path, effects=direct_effects | indirect_effects)

In [21]:
supporting_literals

{('move', ¬battery(2)), ('move', ¬position(0))}

In [22]:
initial_literals

set()

## Third

In [23]:
supporting_events, uncaused_literals = get_third_causal_explanation(action_description=AD, scenario_path=scenario_path, outcome={o}, transition_event="move")

In [24]:
supporting_events

{('move', ¬battery(0)),
 ('move', battery(1)),
 ('move', position(1)),
 ('move', ¬position(2)),
 ('move', position(2))}

In [25]:
uncaused_literals

{battery(2), position(0)}

## Unsuccessfull Route

In [26]:
state1, relevant_stmt1 = AD.execute(initial_state, "wait")

In [27]:
state1

{¬battery(0),
 battery(1),
 ¬battery(2),
 ¬objective_reached,
 position(0),
 ¬position(1),
 ¬position(2)}

In [28]:
relevant_stmt1

[wait causes ¬battery(2) if battery(2),
 wait causes battery(1) if battery(2),
 wait causes position(0) if position(0),
 ¬battery(2) if battery(1),
 ¬position(1) if position(0),
 ¬position(2) if position(0),
 ¬objective_reached if ¬position(2),
 ¬battery(0) if battery(1)]

In [29]:
AD.executable_events(state1, events)

({'move', 'wait'}, ())

In [30]:
state2, relevant_stmt2 = AD.execute(state1, "move")

In [31]:
state2

{battery(0),
 ¬battery(1),
 ¬battery(2),
 ¬objective_reached,
 ¬position(0),
 position(1),
 ¬position(2)}

In [32]:
relevant_stmt2

[move causes ¬battery(1) if battery(1),
 move causes ¬position(0) if position(0),
 move causes position(1) if position(0),
 move causes battery(0) if battery(1),
 ¬battery(1) if battery(0),
 ¬position(2) if position(1),
 ¬position(0) if position(1),
 ¬objective_reached if ¬position(2),
 ¬battery(2) if battery(0)]

In [33]:
AD.executable_events(state2, events)

({'wait'}, (move impossible_if battery(0),))

In [34]:
scenario_path = (initial_state,[("wait", state1),("move", state2)])

In [35]:
direct_effects, indirect_effects = get_first_causal_explanation(action_description=AD, scenario_path=scenario_path, outcome={~o, ~p2}, transition_event="move")

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
direct_effects

In [ ]:
indirect_effects